In [ ]:
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras import models
from keras_tuner.applications import HyperResNet
from keras_tuner.applications import HyperXception
from keras_tuner import  HyperParameters
import tensorflow as tf
import matplotlib


import dataset

In [ ]:
 # read dataset

# Normalize pixel values between 0 and 1
train_images = train_images.reshape((n_train, 256, 256, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((n_test, 256, 256, 1))
test_images = test_images.astype('float32') / 255

#to categorical (?)

In [ ]:
#early stopping callbacks, used for each model
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tensorboard = keras.callbacks.TensorBoard("log_dir")

NUM_CLASSES = 10

In [ ]:
#tune a model given an hypermodel, find best hyperparameters, and return it
def tune_model(hypermodel):

    tuner = keras_tuner.Hyperband( #decide objective and type of tuner
    hypermodel,
    objective='val_accuracy',
    max_epochs=10,
    directory='galaxy_dir',
    project_name='galaxy_classification_project'

    )

    #find best parameters
    tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early, tensorboard])

    # Get the optimal hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    #tune model with optimal parameters
    tuned_model = tuner.hypermodel.build(best_hps)

    return tuned_model


parameter tuning

In [ ]:
class CNNFromScratch(keras_tuner.HyperModel):
    #override build
    def build_model(self,hp):
      model = models.Sequential()

      #add input layer
      model.add(layers.Input(shape=(256,256,1)))

      #add data augmentation layer???

      # Tune the number of layers.
      for i in range(hp.Int("num_layers", min_value = 1, max_value = 64, step = 8)):

          #tune filter size
          hp_filters = hp.Int(f"conv_filter_{i}", min_value=16, max_value=64, step=16)
          #tune filter size. Most used are 3x3 and 5x5
          hp_kernel_size=hp.Choice("conv_kernel", values = [3,5])
          #add convolutional and pooling layer
          model.add(layers.Conv2D(filters = hp_filters, kernel_size = (hp_kernel_size,hp_kernel_size), activation='relu'))
          model.add(layers.MaxPooling2D((2, 2)))

          model.add(layers.BatchNormalization())#da tunare?

      # flatten ed eventuale dense prima (tunare presenza)
      model.add(layers.Flatten())
      if hp.Boolean("dense_layer"):
          model.add(layers.Dense())

      #decide whether to use dropout or not
      if hp.Boolean("dropout"):
          hp_dropout_rate = hp.Int(f"dropout_rate", min_value=0.5, max_value=0.8, step=0.1)
          model.add(layers.Dropout(rate=0.25))

      #add classification layer
      model.add(layers.Dense(10, activation="softmax"))

      #tune learning rate
      hp_learning_rate = hp.Float("learning rate", min_value=1e-4, max_value=1e-2, sampling="log")
      model.compile(
          optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), # tuning optimizer
          loss="categorical_crossentropy",
          metrics=["accuracy"],
      )

      return model

    #override fit
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            epochs=hp.Int("epochs", min_value = 5, max_value = 20, step = 5),
            batch_size=hp.Choice("batch_size", [16, 32]),
            **kwargs)

In [ ]:
#get tuned models
model_cnn = tune_model(CNNFromScratch())
model_resnet = tune_model(HyperResNet(input_shape=(256, 256, 1), classes=NUM_CLASSES))
model_xception = tune_model(HyperXception(input_shape=(256, 256, 1), classes=NUM_CLASSES))

 Can delete from now on I guess(?)

In [ ]:
tuner = keras_tuner.Hyperband( #decide objective and type of tuner
    CNNFromScratch(),
    objective='val_accuracy',
    max_epochs=10,
    directory='galaxy_dir',
    project_name='galaxy_classification_project'

)

#find best parameters
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early, tensorboard])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

#tune model with optimal parameters
tuned_model = tuner.hypermodel.build(best_hps)

Possibile test con entrambi gli approcci ?

In [ ]:
tuner = keras_tuner.Hyperband(
    HyperResNet(input_shape=(256, 256, 1), classes=10), #used a in-built model for tuning, useful in CV applications.
                                                      #there is no need to define a search space so users can use it directly
    objective='val_accuracy', #decide objective
    max_epochs=10,
    directory='galaxy_dir',  #store search result in this dir
    project_name='galaxy_classification_project' # will be a sub dir of the above directory
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tensorboard = keras.callbacks.TensorBoard(log_dir)
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early, tensorboard])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

tuned_model = tuner.hypermodel.build(best_hps)

In [ ]:
tuner = keras_tuner.Hyperband(
    HyperXception(input_shape=(256, 256, 1), classes=10), #used a in-built model for tuning, useful in CV applications.
                                                      #there is no need to define a search space so users can use it directly
    objective='val_accuracy', #decide objective
    max_epochs=10,
    directory='galaxy_dir',  #store search result in this dir
    project_name='galaxy_classification_project' # will be a sub dir of the above directory
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tensorboard = keras.callbacks.TensorBoard(log_dir)
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early, tensorboard])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

tuned_model = tuner.hypermodel.build(best_hps)